# Compute parameters needed for simulation from other datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Set age bins
bins = [(0, 19), (20, 49), (50, 64), (65, 200)]

In [3]:
# Compute the proportions of the population of São Paulo that belog to each bin
census = pd.read_excel("distribuicao_etaria_sp_2010.xlsx", index_col=[0])
census["Total"] = census["Homens"] + census["Mulheres"]

# Create columns with minimal and maximal age in each bin of the original data
def str2ages(str):
    if str.startswith("Mais"):
        return 100, 200
    a_loc = str.find(" a ")
    a = int(str[:a_loc])
    A = int(str[a_loc + 3:a_loc + 5])
    return(a, A)

min_age, max_age = [], []
for age in census.index:
    a, A = str2ages(age)
    min_age.append(a)
    max_age.append(A)
census["Min age"] = min_age
census["Max age"] = max_age

# Calculate the proportion of the population in each of the target bins
pop_by_age = {}
total_pop = census["Total"].sum()
for b in bins:
    mask = np.logical_and(census["Min age"] >= b[0], census["Max age"] <= b[1])
    pop_by_age[b] = census[mask]["Total"].sum() / total_pop
pop_by_age = pd.Series(pop_by_age, name="Population ratios")

# Print the age distribution
pd.options.display.float_format = '{:,.3f}'.format
print("Age distribution in the population")
print(pop_by_age)

Age distribution in the population
0   19    0.295
20  49    0.482
50  64    0.144
65  200   0.078
Name: Population ratios, dtype: float64


Obs: I will add the bin (0, 19) up using 30% to add 1.0

In [4]:
# Compute the age distribution of ICU demand
icu_use = pd.read_csv("20210130_SRAG.csv", sep=';', decimal=",", parse_dates=['Data de Notificação'])
icu_use = icu_use[icu_use["OUTRAS SRAG"] == "COVID 19"]

icu_by_age = {}
for b in bins:
    low, up = b
    icu_by_age[b] = np.logical_and(icu_use["Nu Idade N"] >= low, icu_use["Nu Idade N"] <= up).sum()
icu_by_age = pd.Series(icu_by_age, name="Age distribution") / len(icu_use)

# Print the age ICU distribution
print("ICU need by age")
print(icu_by_age)
print("\n")

# Now compute the ratio of ICU need by individual of each group
print("Correction factor for ICU usage")
print(icu_by_age / pop_by_age)

ICU need by age
0   19    0.016
20  49    0.282
50  64    0.297
65  200   0.405
Name: Age distribution, dtype: float64


Correction factor for ICU usage
0   19    0.055
20  49    0.584
50  64    2.057
65  200   5.164
dtype: float64


In [5]:
# Now agregate the contact matrix

# Read original contact matrix. It has in C_{i, j} = number of people from group i that people
# from group j meet each day. The sum of column of j is the number of enconters a member
# of group j has each day. 
full_contact = pd.read_excel("contact_matrix.xlsx", header=None)
full_contact.set_index(pd.Index(list(5*full_contact.index)), inplace=True)
full_contact.columns = list(5*full_contact.columns)

# Compress the matrix to bins
contact = {}
for b1 in bins:
    contact[b1] = []
    for b2 in bins:
        b1_mask = np.logical_and(full_contact.columns >= b1[0], full_contact.columns < b1[1])
        b2_mask = np.logical_and(full_contact.index >= b2[0], full_contact.index < b2[1])
        contact[b1].append(full_contact.loc[b2_mask, b1_mask].to_numpy().sum())
        print(f"Numero of elements of {b2} that {b1} meets is", contact[b1][-1])
contact = pd.DataFrame(contact, index=contact.keys())
print("Absolute contact")
print(contact.T)
print("\n")
contact = contact/contact.sum()
contact = contact.T
print("Relative contact")
print(contact)

Numero of elements of (0, 19) that (0, 19) meets is 50.98675790128427
Numero of elements of (20, 49) that (0, 19) meets is 24.107067294185118
Numero of elements of (50, 64) that (0, 19) meets is 9.009466158036828
Numero of elements of (65, 200) that (0, 19) meets is 5.410516993713345
Numero of elements of (0, 19) that (20, 49) meets is 21.061503081752065
Numero of elements of (20, 49) that (20, 49) meets is 62.95736379317733
Numero of elements of (50, 64) that (20, 49) meets is 16.043085058072208
Numero of elements of (65, 200) that (20, 49) meets is 6.4292960331042375
Numero of elements of (0, 19) that (50, 64) meets is 2.8121249425288437
Numero of elements of (20, 49) that (50, 64) meets is 8.45247660687524
Numero of elements of (50, 64) that (50, 64) meets is 5.035690092551668
Numero of elements of (65, 200) that (50, 64) meets is 2.1762279491399545
Numero of elements of (0, 19) that (65, 200) meets is 0.7317890032102449
Numero of elements of (20, 49) that (65, 200) meets is 0.97446